# Import Libraries

In [1]:
import os
import torch
from Preprocess import Process
from NeuralNetworkDIY import ClassifierNetwork, Model
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image

print(torch.cuda.is_available())

True


# Parameters for the dataset

In [2]:
RAW_DIR = "data/raw/140k face"
PROCESSED_DIR = "data/processed"
IMAGE_SIZE=(128,128)

# Image Preprocessing

In [3]:
process=Process(RAW_DIR, PROCESSED_DIR, IMAGE_SIZE, delete_process_dir=True, batch_size=1000, aug_amount=0)
process.run(0.7, 0.3, 1)

Working on fake


Processing valid: 100%|██████████| 48/48 [10:42<00:00, 13.39s/it]


Working on real


Processing valid: 100%|██████████| 21/21 [03:49<00:00, 10.92s/it]


# DataLoader and Custom Dataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        transform = transforms.ToTensor()
        
        img= transform(img)
        # Extract label from the filename (fake -> 1, real -> 0)
        label = 1 if "fake" in os.path.basename(img_path).lower() else 0

        return img, label, img_path

In [5]:
train_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/train/rgb')
valid_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/valid/rgb')

train_loader = DataLoader(train_data, batch_size=1000, shuffle=True) #batchsize set to 1000, GPU can handle it, maybe more too
valid_loader = DataLoader(valid_data, batch_size=1000, shuffle=False)

# Creating the network, model, training, predicting

In [6]:
network=ClassifierNetwork(IMAGE_SIZE)

In [7]:
model=Model(network, 20)

In [ ]:
model.train(train_loader)

Epoch 1/20, Loss: 1.1753672527970735
Epoch 2/20, Loss: 0.6025866880920363
Epoch 3/20, Loss: 0.5447155918393817
Epoch 4/20, Loss: 0.46101849327176253
Epoch 5/20, Loss: 0.36976973869785756
Epoch 6/20, Loss: 0.2761201551241904


In [ ]:
results=model.predict(valid_loader)